In [17]:
import pandas as pd
import os

curr_dir = os.getcwd()

parent_dir = os.path.dirname(curr_dir) # gtes the name of the parent directory

singles_net_stats_path = os.path.join(parent_dir, 'stats', 'singles_net_stats', 'singles_net_stats2.csv')
df = pd.read_csv(singles_net_stats_path)

In [18]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [19]:
# columns that need to be encoded
str_vals = ['tourney_name', 'surface','tourney_level','winner_hand','winner_ioc','loser_hand','loser_ioc','round','winner_name','loser_name','winner_entry','loser_entry']
# array of encoded label encoder objs
label_encoder_variables = []
for variable_name in str_vals:
    var_name = "nlabel_" + variable_name
    globals()[var_name] = LabelEncoder()
    label_encoder_variables.append(globals()[var_name])

In [20]:
label_encoder_variables

[LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder()]

In [21]:
# 1
def encode():
    for i in range(len(str_vals)):
        if str_vals[i] in df.columns:
            encoded_number = label_encoder_variables[i].fit_transform(df[str_vals[i]]) # encodes each needed column --> type df
            df[str_vals[i] + "_n"] = encoded_number
    df.to_csv(singles_net_stats_path)


In [22]:
encode()

In [32]:
# 2
def build_player_name_map():
    player_name_map = defaultdict(int)
    
    winners = zip(df['winner_name'], df['winner_name_n'])
    losers = zip(df['loser_name'], df['loser_name_n'])

    for name, encoded in list(winners) + list(losers):
        player_name_map[name] = encoded

    return player_name_map

In [24]:
# 3 
def drop_cols():
    # drop str value columns plus winner name and loser name
    global df
    for col in str_vals: 
        if col in df.columns:
            df = df.drop(str_vals,axis='columns')
            temp_cols = ['score','Unnamed: 0.1','Unnamed: 0']
            for temp_cols1 in temp_cols:
                if temp_cols1 in df.columns:
                    df = df.drop(['winner_rank','loser_rank','score','Unnamed: 0.1','Unnamed: 0'],axis='columns')

In [33]:
# rank diff --> negative = loser rank points > winner rank points --> positive = winner rank points > loser rank points
df['rank_points_diff'] = df['winner_rank_points'] - df['loser_rank_points']
df

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import warnings
import joblib
warnings.filterwarnings('ignore')

In [29]:
y = df['winner_name_n']
X = []
for column_name in list(df.columns):
    if column_name != 'winner_name_n' and column_name != 'loser_name_n':
        X.append(column_name)
X = df[X]
X.drop(str_vals,axis='columns')

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,draw_size,winner_seed,winner_ht,winner_age,loser_seed,loser_ht,loser_age,...,surface_n,tourney_level_n,winner_hand_n,winner_ioc_n,loser_hand_n,loser_ioc_n,round_n,winner_entry_n,loser_entry_n,rank_points_diff
0,0,0,0,32.0,NaN,NaN,NaN,NaN,NaN,24.0,...,2,0,3,5,3,61,6,10,11,NaN
1,1,1,1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0,2,98,3,61,6,10,11,NaN
2,2,2,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0,3,54,3,61,6,10,11,NaN
3,3,3,3,32.0,NaN,178.0,24.3,NaN,NaN,NaN,...,2,0,2,81,3,145,6,10,11,NaN
4,4,4,4,32.0,NaN,NaN,21.3,NaN,NaN,NaN,...,2,0,2,124,3,61,6,10,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194991,194991,194991,194991,4.0,NaN,NaN,18.8,NaN,188.0,21.8,...,1,1,3,123,2,82,8,10,11,-26.0
194992,194992,194992,194992,4.0,NaN,175.0,26.9,NaN,NaN,23.2,...,3,1,2,127,2,109,8,10,11,35.0
194993,194993,194993,194993,4.0,NaN,185.0,27.3,NaN,183.0,26.4,...,3,1,2,98,2,141,8,10,11,NaN
194994,194994,194994,194994,4.0,NaN,175.0,26.9,NaN,185.0,27.3,...,3,1,2,127,2,109,8,10,11,-42.0


In [30]:
# 4
def train_model():
    global X
    global y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    random_forest_classifier = RandomForestClassifier(n_estimators=100,criterion='entropy',max_depth=13,n_jobs=1,random_state=42)
    random_forest_classifier.fit(X_train,y_train)
    joblib.dump(random_forest_classifier, 'rf_model.pkl')
    return random_forest_classifier

In [ ]:
if __name__ == "__main__":
    train_model()

   Unnamed: 0.7  Unnamed: 0.6  Unnamed: 0.5  Unnamed: 0.4  Unnamed: 0.3  \
0             0             0             0             0             0   
1             1             1             1             1             1   
2             2             2             2             2             2   
3             3             3             3             3             3   
4             4             4             4             4             4   

   Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  draw_size  winner_seed  ...  \
0             0             0           0       32.0          NaN  ...   
1             1             1           1       32.0          NaN  ...   
2             2             2           2       32.0          NaN  ...   
3             3             3           3       32.0          NaN  ...   
4             4             4           4       32.0          NaN  ...   

   winner_hand_n  winner_ioc_n  loser_hand_n  loser_ioc_n  round_n  \
0              3             5    